In [27]:
import pandas as pd
import requests
import math
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import nan 

In [28]:
df_variants = pd.read_csv('bus_variants.csv')
df_stop_features = pd.read_csv('stop_features.csv')
df_destinations = pd.read_csv('Bus_Destinations.csv')
df_routes = pd.read_csv('Routes.csv')
df_stops = pd.read_csv('Bus_Stops.csv')

# Renaming Columns and Preparing Data for Merging

In [29]:
# Dropping uncecessary columns

df_variants.drop(columns=['Unnamed: 0', 'description'], inplace=True)
df_variants.rename(columns={"key": "Bus_Variant", "name": "Route_Name", "Route": "Route_Number"}, inplace=True)

# Creating a new feature which will have the number of variants running on Transit for each route.
df_variants['Variants_Count'] = df_variants.groupby(['Route_Number'])["Route_Number"].transform("count")

df_variants

,Bus_Variant,Route_Name,Route_Number,Variants_Count
0,11-1-M,Portage-Kildonan to City Hall,11.0,10
1,11-1-D,Portage-Kildonan to North Kildonan via Donwood,11.0,10
2,11-0-P,Portage-Kildonan to Polo Park,11.0,10
3,11-1-R,Portage-Kildonan to North Kildonan via Rothesay,11.0,10
4,11-1-G,Portage-Kildonan to Glenway via,11.0,10
...,...,...,...,...
280,97-1-P,to Point Douglas,97.0,3
281,97-0-F,to Fife,97.0,3
282,97-1-f,to Fife,97.0,3
283,93-1-S,to St. Vital Centre,93.0,2


In [30]:
df_stop_features.rename(columns={"name": "Feature", "count": "Count"}, inplace=True)

# Creating a column to count number of features in each stop
df_stop_features['Types_of_Features_Count']= df_stop_features.groupby(["Stop_Number"])['Count'].transform("count")
df_stop_features = df_stop_features.drop(columns=['Feature', 'Count']).drop_duplicates()

In [31]:
df_destinations.drop(columns=['Unnamed: 0', 'key'], inplace=True)
df_destinations.rename(columns={"name": "Destination_Name"}, inplace=True)
df_destinations

,Destination_Name,Bus_Variant
0,City Hall,45-0-K
1,Point Douglas,45-0-K
2,Kildonan Place,45-0-K
3,Munroe,45-0-K
4,Crossroads Station,45-0-K
...,...,...
1032,Downtown,23-1-C
1033,City Hall,11-1-M
1034,Polo Park,11-1-M
1035,Polo Park,11-1-M


In [32]:
df_routes['customer-type'].value_counts()

regular    84
Name: customer-type, dtype: int64

In [33]:
df_routes = df_routes[['number', 'name', 'coverage']]
df_routes.rename(columns={"name": "Route_Name", "number": "Route_Number"}, inplace=True)
df_routes

,Route_Number,Route_Name,coverage
0,11,Route 11 Portage-Kildonan,regular
1,15,Route 15 Sargent-Mountain,regular
2,16,Route 16 Selkirk-Osborne,regular
3,17,Route 17 McGregor,regular
4,12,Route 12 William,regular
...,...,...,...
79,90,Route 90 Kildonan Place - Whellams Lane,regular
80,93,Route 93 South St. Vital - St. Marys,regular
81,96,Route 96 St. Vital Centre - Windsor Park,regular
82,92,Route 92 Crossroads Station - Lakeside Meadows,regular


In [34]:
df_stops.drop(columns=['Unnamed: 0', 'key', 'centre.utm.zone', 'street.leg', 'cross-street.leg' ], inplace=True)
df_stops.rename(columns={"name": "Stop_Name", "number": "Stop_Number"}, inplace=True)

# Getting a variable number of bus variants serviced at each stop. We can have info about number of buses stopping at each stop
df_stops['Variants_Serviced_Count'] = df_stops.groupby(["Stop_Number"])['Bus_Variant'].transform('count')
df_stops['Number_of_Stoppages'] = df_stops.groupby(['Bus_Variant'])['Stop_Number'].transform('count')
df_stops

,Stop_Name,Stop_Number,direction,side,street.key,street.name,street.type,cross-street.key,cross-street.name,cross-street.type,centre.utm.x,centre.utm.y,centre.geographic.latitude,centre.geographic.longitude,Bus_Variant,Variants_Serviced_Count,Number_of_Stoppages
0,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,635333.0,5529475.0,49.90234,-97.11540,45-0-K,7.0,57
1,Eastbound Higgins at McArthur,10354.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2387.0,McArthur Street,Street,635217.0,5529410.0,49.90178,-97.11704,45-0-K,7.0,57
2,Eastbound Higgins at Acores,10355.0,Eastbound,Farside Opposite,1736.0,Higgins Avenue,Avenue,4755.0,Acores Street,Street,634999.0,5529448.0,49.90217,-97.12006,45-0-K,7.0,57
3,Eastbound Higgins at Annabella,10358.0,Eastbound,Farside,1736.0,Higgins Avenue,Avenue,110.0,Annabella Street,Street,634845.0,5529481.0,49.90250,-97.12219,45-0-K,7.0,57
4,Eastbound Graham at Fort (Wpg Square),10611.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,1365.0,Fort Street,Street,633667.0,5528384.0,49.89290,-97.13897,45-0-K,31.0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15358,Northbound Main at McDermot,10628.0,Northbound,Nearside,2265.0,Main Street,Street,2400.0,McDermot Avenue,Avenue,633682.0,5528839.0,49.89699,-97.13860,11-1-M,41.0,28
15359,Northbound Main at James (Concert Hall),10629.0,Northbound,Nearside Opposite,2265.0,Main Street,Street,1862.0,James Avenue,Avenue,633730.0,5529165.0,49.89991,-97.13782,11-1-M,41.0,28
15360,Eastbound Pacific at Martha,10800.0,Eastbound,Nearside,2734.0,Pacific Avenue,Avenue,2351.0,Martha Street,Street,633877.0,5529255.0,49.90069,-97.13574,11-1-M,29.0,28
15361,Southbound St James at Ness South,20001.0,Southbound,Farside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,629189.0,5527072.0,49.88209,-97.20172,11-1-M,7.0,28


In [35]:
df_stops

,Stop_Name,Stop_Number,direction,side,street.key,street.name,street.type,cross-street.key,cross-street.name,cross-street.type,centre.utm.x,centre.utm.y,centre.geographic.latitude,centre.geographic.longitude,Bus_Variant,Variants_Serviced_Count,Number_of_Stoppages
0,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,635333.0,5529475.0,49.90234,-97.11540,45-0-K,7.0,57
1,Eastbound Higgins at McArthur,10354.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2387.0,McArthur Street,Street,635217.0,5529410.0,49.90178,-97.11704,45-0-K,7.0,57
2,Eastbound Higgins at Acores,10355.0,Eastbound,Farside Opposite,1736.0,Higgins Avenue,Avenue,4755.0,Acores Street,Street,634999.0,5529448.0,49.90217,-97.12006,45-0-K,7.0,57
3,Eastbound Higgins at Annabella,10358.0,Eastbound,Farside,1736.0,Higgins Avenue,Avenue,110.0,Annabella Street,Street,634845.0,5529481.0,49.90250,-97.12219,45-0-K,7.0,57
4,Eastbound Graham at Fort (Wpg Square),10611.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,1365.0,Fort Street,Street,633667.0,5528384.0,49.89290,-97.13897,45-0-K,31.0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15358,Northbound Main at McDermot,10628.0,Northbound,Nearside,2265.0,Main Street,Street,2400.0,McDermot Avenue,Avenue,633682.0,5528839.0,49.89699,-97.13860,11-1-M,41.0,28
15359,Northbound Main at James (Concert Hall),10629.0,Northbound,Nearside Opposite,2265.0,Main Street,Street,1862.0,James Avenue,Avenue,633730.0,5529165.0,49.89991,-97.13782,11-1-M,41.0,28
15360,Eastbound Pacific at Martha,10800.0,Eastbound,Nearside,2734.0,Pacific Avenue,Avenue,2351.0,Martha Street,Street,633877.0,5529255.0,49.90069,-97.13574,11-1-M,29.0,28
15361,Southbound St James at Ness South,20001.0,Southbound,Farside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,629189.0,5527072.0,49.88209,-97.20172,11-1-M,7.0,28


# Data Merging

In [36]:
# Merging df_stops and df_stop features on stop number

df_stop_features_merged = df_stops.merge(df_stop_features, on='Stop_Number', how='left' )
df_stop_features_merged

,Stop_Name,Stop_Number,direction,side,street.key,street.name,street.type,cross-street.key,cross-street.name,cross-street.type,centre.utm.x,centre.utm.y,centre.geographic.latitude,centre.geographic.longitude,Bus_Variant,Variants_Serviced_Count,Number_of_Stoppages,Types_of_Features_Count
0,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,635333.0,5529475.0,49.90234,-97.11540,45-0-K,7.0,57,0.0
1,Eastbound Higgins at McArthur,10354.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2387.0,McArthur Street,Street,635217.0,5529410.0,49.90178,-97.11704,45-0-K,7.0,57,0.0
2,Eastbound Higgins at Acores,10355.0,Eastbound,Farside Opposite,1736.0,Higgins Avenue,Avenue,4755.0,Acores Street,Street,634999.0,5529448.0,49.90217,-97.12006,45-0-K,7.0,57,0.0
3,Eastbound Higgins at Annabella,10358.0,Eastbound,Farside,1736.0,Higgins Avenue,Avenue,110.0,Annabella Street,Street,634845.0,5529481.0,49.90250,-97.12219,45-0-K,7.0,57,0.0
4,Eastbound Graham at Fort (Wpg Square),10611.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,1365.0,Fort Street,Street,633667.0,5528384.0,49.89290,-97.13897,45-0-K,31.0,57,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15358,Northbound Main at McDermot,10628.0,Northbound,Nearside,2265.0,Main Street,Street,2400.0,McDermot Avenue,Avenue,633682.0,5528839.0,49.89699,-97.13860,11-1-M,41.0,28,2.0
15359,Northbound Main at James (Concert Hall),10629.0,Northbound,Nearside Opposite,2265.0,Main Street,Street,1862.0,James Avenue,Avenue,633730.0,5529165.0,49.89991,-97.13782,11-1-M,41.0,28,3.0
15360,Eastbound Pacific at Martha,10800.0,Eastbound,Nearside,2734.0,Pacific Avenue,Avenue,2351.0,Martha Street,Street,633877.0,5529255.0,49.90069,-97.13574,11-1-M,29.0,28,0.0
15361,Southbound St James at Ness South,20001.0,Southbound,Farside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,629189.0,5527072.0,49.88209,-97.20172,11-1-M,7.0,28,1.0


In [37]:
# Df_Variants and Df_Routes merge

df_routes_variants_merged = df_variants.merge(df_routes, how='left', on='Route_Number')
df_routes_variants_merged.rename(columns={"Route_Name_x": "Route_Name", "Route_Name_y": "Route_Description"}, inplace=True)
df_routes_variants_merged

,Bus_Variant,Route_Name,Route_Number,Variants_Count,Route_Description,coverage
0,11-1-M,Portage-Kildonan to City Hall,11.0,10,Route 11 Portage-Kildonan,regular
1,11-1-D,Portage-Kildonan to North Kildonan via Donwood,11.0,10,Route 11 Portage-Kildonan,regular
2,11-0-P,Portage-Kildonan to Polo Park,11.0,10,Route 11 Portage-Kildonan,regular
3,11-1-R,Portage-Kildonan to North Kildonan via Rothesay,11.0,10,Route 11 Portage-Kildonan,regular
4,11-1-G,Portage-Kildonan to Glenway via,11.0,10,Route 11 Portage-Kildonan,regular
...,...,...,...,...,...,...
280,97-1-P,to Point Douglas,97.0,3,Route 97 Inkster Industrial Park - Point Douglas,regular
281,97-0-F,to Fife,97.0,3,Route 97 Inkster Industrial Park - Point Douglas,regular
282,97-1-f,to Fife,97.0,3,Route 97 Inkster Industrial Park - Point Douglas,regular
283,93-1-S,to St. Vital Centre,93.0,2,Route 93 South St. Vital - St. Marys,regular


In [38]:
# Putting all the dataframe together

df = df_stop_features_merged.merge(df_destinations, on='Bus_Variant', how='left').merge(df_routes_variants_merged, on='Bus_Variant', how='left')
df

,Stop_Name,Stop_Number,direction,side,street.key,street.name,street.type,cross-street.key,cross-street.name,cross-street.type,...,Bus_Variant,Variants_Serviced_Count,Number_of_Stoppages,Types_of_Features_Count,Destination_Name,Route_Name,Route_Number,Variants_Count,Route_Description,coverage
0,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,...,45-0-K,7.0,57,0.0,City Hall,Talbot to Kildonan Place,45.0,3,Route 45 Talbot,regular
1,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,...,45-0-K,7.0,57,0.0,Point Douglas,Talbot to Kildonan Place,45.0,3,Route 45 Talbot,regular
2,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,...,45-0-K,7.0,57,0.0,Kildonan Place,Talbot to Kildonan Place,45.0,3,Route 45 Talbot,regular
3,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,...,45-0-K,7.0,57,0.0,Munroe,Talbot to Kildonan Place,45.0,3,Route 45 Talbot,regular
4,Eastbound Higgins at Point Douglas,10351.0,Eastbound,Nearside,1736.0,Higgins Avenue,Avenue,2889.0,Point Douglas Avenue,Avenue,...,45-0-K,7.0,57,0.0,Crossroads Station,Talbot to Kildonan Place,45.0,3,Route 45 Talbot,regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77432,Southbound St James at Ness South,20001.0,Southbound,Farside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,...,11-1-M,7.0,28,1.0,Downtown,Portage-Kildonan to City Hall,11.0,10,Route 11 Portage-Kildonan,regular
77433,Southbound St James at Ness North,20002.0,Southbound,Nearside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,...,11-1-M,5.0,28,1.0,City Hall,Portage-Kildonan to City Hall,11.0,10,Route 11 Portage-Kildonan,regular
77434,Southbound St James at Ness North,20002.0,Southbound,Nearside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,...,11-1-M,5.0,28,1.0,Polo Park,Portage-Kildonan to City Hall,11.0,10,Route 11 Portage-Kildonan,regular
77435,Southbound St James at Ness North,20002.0,Southbound,Nearside,3444.0,St James Street,Street,2607.0,Ness Avenue,Avenue,...,11-1-M,5.0,28,1.0,Polo Park,Portage-Kildonan to City Hall,11.0,10,Route 11 Portage-Kildonan,regular


In [39]:
#df.to_csv('Test_Merged_All.csv', index=False)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77437 entries, 0 to 77436
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Stop_Name                    77434 non-null  object 
 1   Stop_Number                  77434 non-null  float64
 2   direction                    77434 non-null  object 
 3   side                         77070 non-null  object 
 4   street.key                   77434 non-null  float64
 5   street.name                  77434 non-null  object 
 6   street.type                  76628 non-null  object 
 7   cross-street.key             77434 non-null  float64
 8   cross-street.name            77434 non-null  object 
 9   cross-street.type            76686 non-null  object 
 10  centre.utm.x                 77434 non-null  float64
 11  centre.utm.y                 77434 non-null  float64
 12  centre.geographic.latitude   77434 non-null  float64
 13  centre.geographi

In [41]:
df.nunique()

Stop_Name                      5072
Stop_Number                    5084
direction                         4
side                              5
street.key                      496
street.name                     482
street.type                      20
cross-street.key               1798
cross-street.name              1764
cross-street.type                28
centre.utm.x                   4316
centre.utm.y                   4381
centre.geographic.latitude     4261
centre.geographic.longitude    4510
Bus_Variant                     258
Variants_Serviced_Count          33
Number_of_Stoppages             100
Types_of_Features_Count           6
Destination_Name                155
Route_Name                      241
Route_Number                     83
Variants_Count                    9
Route_Description                83
coverage                          4
dtype: int64

In [42]:
df.coverage.value_counts()

regular          56344
express          15997
feeder            2859
super express     2237
Name: coverage, dtype: int64

In [43]:
df_stop_schedule = pd.read_csv('Stop_Schedule.csv')
df_stop_schedule

,Stop_Number,key,cancelled,times.departure.scheduled,times.departure.estimated,variant.key,variant.name,bus.key,bus.bike-rack,bus.wifi,times.arrival.scheduled,times.arrival.estimated,Delay_Time,IsDelayed
0,10830,22722773-2,False,2022-12-26T13:03:37,2022-12-26T13:03:37,11-1-D,Portage-Kildonan to North Kildonan via Donwood,191.0,False,False,2022-12-26T13:03:37,2022-12-26T13:03:37,0.0,False
1,60776,22721899-46,False,2022-12-26T13:22:56,2022-12-26T13:22:56,38-1-F,Salter to The Forks,419.0,True,False,2022-12-26T13:22:56,2022-12-26T13:22:56,0.0,False
2,50794,22722776-49,False,2022-12-26T13:09:00,2022-12-26T13:09:00,11-0-C,Portage-Kildonan to Crestview via,192.0,False,False,2022-12-26T13:09:00,2022-12-26T13:09:00,0.0,False
3,60776,22721004-31,False,2022-12-26T11:41:12,2022-12-26T11:41:12,650-1-S,McGillivray to Seel Station,180.0,False,False,2022-12-26T11:41:12,2022-12-26T11:41:12,0.0,False
4,30809,22721224-29,False,2022-12-26T12:03:08,2022-12-26T12:03:08,89-1-K,to Kildonan Place,808.0,False,False,2022-12-26T12:03:08,2022-12-26T12:03:08,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,30563,22722767-95,False,2022-12-26T12:19:16,2022-12-26T12:19:16,11-1-D,Portage-Kildonan to North Kildonan via Donwood,192.0,False,False,2022-12-26T12:19:16,2022-12-26T12:19:16,0.0,False
99996,50430,22722131-78,False,2022-12-26T12:43:52,2022-12-26T12:43:52,20-0-W,Watt-Academy to Watt & Leighton,129.0,False,False,2022-12-26T12:43:52,2022-12-26T12:43:52,0.0,False
99997,10022,22722774-15,False,2022-12-26T12:11:34,2022-12-26T12:11:34,11-0-S,Portage-Kildonan to St. Charles via,300.0,False,False,2022-12-26T12:11:34,2022-12-26T12:11:34,0.0,False
99998,10118,22722297-73,False,2022-12-26T12:03:00,2022-12-26T12:03:00,18-1-A,North Main-Corydon to Assiniboine Park,345.0,False,False,2022-12-26T12:03:00,2022-12-26T12:03:00,0.0,False


In [44]:
# Merging schedule data

df_final = df.merge(df_stop_schedule, on='Stop_Number', how='inner')
df_final

,Stop_Name,Stop_Number,direction,side,street.key,street.name,street.type,cross-street.key,cross-street.name,cross-street.type,...,times.departure.estimated,variant.key,variant.name,bus.key,bus.bike-rack,bus.wifi,times.arrival.scheduled,times.arrival.estimated,Delay_Time,IsDelayed
0,Eastbound Graham at Smith,10615.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,3371.0,Smith Street,Street,...,2022-12-26T12:24:00,45-0-K,Talbot to Kildonan Place,359.0,False,False,2022-12-26T12:24:00,2022-12-26T12:24:00,0.0,False
1,Eastbound Graham at Smith,10615.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,3371.0,Smith Street,Street,...,2022-12-26T12:24:00,45-0-K,Talbot to Kildonan Place,359.0,False,False,2022-12-26T12:24:00,2022-12-26T12:24:00,0.0,False
2,Eastbound Graham at Smith,10615.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,3371.0,Smith Street,Street,...,2022-12-26T12:24:00,45-0-K,Talbot to Kildonan Place,359.0,False,False,2022-12-26T12:24:00,2022-12-26T12:24:00,0.0,False
3,Eastbound Graham at Smith,10615.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,3371.0,Smith Street,Street,...,2022-12-26T12:24:00,45-0-K,Talbot to Kildonan Place,359.0,False,False,2022-12-26T12:24:00,2022-12-26T12:24:00,0.0,False
4,Eastbound Graham at Smith,10615.0,Eastbound,Nearside,1533.0,Graham Avenue,Avenue,3371.0,Smith Street,Street,...,2022-12-26T12:24:00,45-0-K,Talbot to Kildonan Place,359.0,False,False,2022-12-26T12:24:00,2022-12-26T12:24:00,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806410,Southbound Buchanan at Peltier,20439.0,Southbound,Nearside Opposite,547.0,Buchanan Boulevard,Boulevard,2817.0,Peltier Avenue,Avenue,...,2022-12-26T12:13:35,BLUE-1-D,BLUE to Downtown,376.0,True,False,2022-12-26T12:13:35,2022-12-26T12:13:35,0.0,False
806411,Southbound Buchanan at Peltier,20439.0,Southbound,Nearside Opposite,547.0,Buchanan Boulevard,Boulevard,2817.0,Peltier Avenue,Avenue,...,2022-12-26T13:28:26,18-0-J,North Main-Corydon to Garden City Centre via J...,778.0,False,False,2022-12-26T13:28:26,2022-12-26T13:28:26,0.0,False
806412,Southbound Buchanan at Peltier,20439.0,Southbound,Nearside Opposite,547.0,Buchanan Boulevard,Boulevard,2817.0,Peltier Avenue,Avenue,...,2022-12-26T13:05:22,24-1-P,Ness Express to Polo Park,837.0,False,False,2022-12-26T13:05:22,2022-12-26T13:05:22,0.0,False
806413,Southbound Buchanan at Peltier,20439.0,Southbound,Nearside Opposite,547.0,Buchanan Boulevard,Boulevard,2817.0,Peltier Avenue,Avenue,...,2022-12-26T12:20:00,33-1-D2,to Downtown,336.0,False,False,2022-12-26T12:20:00,2022-12-26T12:20:00,0.0,False


In [45]:
df_final['street.type'].value_counts()

Avenue       214118
Road         203893
Street       130691
Boulevard    121365
Drive         67847
Row           20800
Crescent      10659
Place          8910
Loop           8212
Terminal       6997
Highway        1960
Way            1952
Bay            1668
Lane           1431
Parkway         642
Freeway          58
Gate              1
Name: street.type, dtype: int64

In [46]:
df_final.nunique()

Stop_Name                       1487
Stop_Number                     1491
direction                          4
side                               5
street.key                       294
street.name                      290
street.type                       17
cross-street.key                 727
cross-street.name                724
cross-street.type                 21
centre.utm.x                    1427
centre.utm.y                    1408
centre.geographic.latitude      1376
centre.geographic.longitude     1447
Bus_Variant                      242
Variants_Serviced_Count           22
Number_of_Stoppages               97
Types_of_Features_Count            5
Destination_Name                 150
Route_Name                       225
Route_Number                      81
Variants_Count                     9
Route_Description                 81
coverage                           4
key                            16008
cancelled                          1
times.departure.scheduled       7389
t

In [47]:
df_final['bus.bike-rack'].value_counts()

False    668753
True     137662
Name: bus.bike-rack, dtype: int64

In [48]:
# Creating another dataframe for ML by removing unique identifiers and unrequired columns which are hihgly cardinal
df_ml = df_final.drop(columns=['bus.wifi', 
                                'times.arrival.scheduled',
                                'times.arrival.estimated',
                                'bus.key', 
                                'variant.key', 
                                'variant.name', 
                                'times.departure.scheduled', 
                                'times.departure.estimated',
                                'cancelled', 
                                'key', 
                                'Route_Name', 
                                'Destination_Name', 
                                'street.key', 
                               'street.name', 
                                'cross-street.key', 
                                'cross-street.name', 
                                'centre.utm.x', 
                                'centre.utm.y', 
                                'cross-street.type', 
                                'Route_Description',
                                'Bus_Variant',
                              'Stop_Name'
                                ])


In [49]:
df_ml

,Stop_Number,direction,side,street.type,centre.geographic.latitude,centre.geographic.longitude,Variants_Serviced_Count,Number_of_Stoppages,Types_of_Features_Count,Route_Number,Variants_Count,coverage,bus.bike-rack,Delay_Time,IsDelayed
0,10615.0,Eastbound,Nearside,Avenue,49.89232,-97.14152,30.0,57,3.0,45.0,3,regular,False,0.0,False
1,10615.0,Eastbound,Nearside,Avenue,49.89232,-97.14152,30.0,57,3.0,45.0,3,regular,False,0.0,False
2,10615.0,Eastbound,Nearside,Avenue,49.89232,-97.14152,30.0,57,3.0,45.0,3,regular,False,0.0,False
3,10615.0,Eastbound,Nearside,Avenue,49.89232,-97.14152,30.0,57,3.0,45.0,3,regular,False,0.0,False
4,10615.0,Eastbound,Nearside,Avenue,49.89232,-97.14152,30.0,57,3.0,45.0,3,regular,False,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806410,20439.0,Southbound,Nearside Opposite,Boulevard,49.89419,-97.31560,1.0,72,0.0,83.0,6,regular,True,0.0,False
806411,20439.0,Southbound,Nearside Opposite,Boulevard,49.89419,-97.31560,1.0,72,0.0,83.0,6,regular,False,0.0,False
806412,20439.0,Southbound,Nearside Opposite,Boulevard,49.89419,-97.31560,1.0,72,0.0,83.0,6,regular,False,0.0,False
806413,20439.0,Southbound,Nearside Opposite,Boulevard,49.89419,-97.31560,1.0,72,0.0,83.0,6,regular,False,0.0,False


In [50]:
#pip install pandas-profiling

In [51]:
#from pandas_profiling import ProfileReport

#design_report = ProfileReport(df_final)
#design_report.to_file(output_file='report.html')

In [53]:
df_ml.to_csv('Data_For_ML.csv', index=False)